# CS-4824 Intro to Machine Learning
# Rohan Jaggannagari
# Anime Machine Learning
Note: sources.txt shows which sources were used for this project

## Imports

In [1]:
import pandas as pd
from sklearn.cluster import KMeans

## Read in the data
I read in the data that was cleaned from the other script.

In [2]:
anime_df = pd.read_csv("clean_data/clean_anime_data.csv")
pd.set_option('display.max_columns', 150) # Wanted to see all of the columns

# Getting rid of these columns as of now, but may add them back if I can process them in a proper way
drop_columns = ['English name', 'Synopsis', 'Duration', 'Unnamed: 0', 'Producers', 'Studios']
anime_df = anime_df.drop(columns = drop_columns)
anime_df.set_index('Name', inplace = True)

# Get rid of row with null values
anime_df = anime_df.dropna()

anime_df.head(5)

,Score,Genres,Type,Episodes,Licensors,Source,Rating
Name,,,,,,,
Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,"Funimation, Bandai Entertainment",Original,R - 17+ (violence & profanity)
Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi",Movie,1.0,Sony Pictures Entertainment,Original,R - 17+ (violence & profanity)
Trigun,8.22,"Action, Adventure, Sci-Fi",TV,26.0,"Funimation, Geneon Entertainment USA",Manga,PG-13 - Teens 13 or older
Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,"Funimation, Bandai Entertainment",Original,PG-13 - Teens 13 or older
Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,Illumitoon Entertainment,Manga,PG - Children


## Data Processing

### Make episodes all int and reducing their weights
I need to get rid of any of the rows that do not have episodes since that goes into my machine learning algorithm. An alternative could have been to give those rows a standardized number, but I decided against that. Also, I need to reduce the weights of the episodes since it is calculated using the euclidean distance. 

In [3]:
anime_df['Episodes'] = anime_df['Episodes'].astype(float)
anime_df['Episodes'] = anime_df['Episodes'] * 0.05

anime_df.head(5)

,Score,Genres,Type,Episodes,Licensors,Source,Rating
Name,,,,,,,
Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,1.30,"Funimation, Bandai Entertainment",Original,R - 17+ (violence & profanity)
Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi",Movie,0.05,Sony Pictures Entertainment,Original,R - 17+ (violence & profanity)
Trigun,8.22,"Action, Adventure, Sci-Fi",TV,1.30,"Funimation, Geneon Entertainment USA",Manga,PG-13 - Teens 13 or older
Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,1.30,"Funimation, Bandai Entertainment",Original,PG-13 - Teens 13 or older
Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",TV,2.60,Illumitoon Entertainment,Manga,PG - Children


### Increase the weight for the score
I want to increase the weight for the score so that it is weighted more.

In [4]:
anime_df['Score'] = anime_df['Score'] * 2.5
anime_df.head(5)

,Score,Genres,Type,Episodes,Licensors,Source,Rating
Name,,,,,,,
Cowboy Bebop,21.875,"Action, Award Winning, Sci-Fi",TV,1.30,"Funimation, Bandai Entertainment",Original,R - 17+ (violence & profanity)
Cowboy Bebop: Tengoku no Tobira,20.950,"Action, Sci-Fi",Movie,0.05,Sony Pictures Entertainment,Original,R - 17+ (violence & profanity)
Trigun,20.550,"Action, Adventure, Sci-Fi",TV,1.30,"Funimation, Geneon Entertainment USA",Manga,PG-13 - Teens 13 or older
Witch Hunter Robin,18.125,"Action, Drama, Mystery, Supernatural",TV,1.30,"Funimation, Bandai Entertainment",Original,PG-13 - Teens 13 or older
Bouken Ou Beet,17.350,"Adventure, Fantasy, Supernatural",TV,2.60,Illumitoon Entertainment,Manga,PG - Children


### Encoding ratings
Ratings is a scale so we would want to start all ages as 0 and R+ as higher so the recomendation system accounts for that. I could have used a label encoder, but it would have randomly assigned those values and I needed there to be a followed pattern for it.

In [5]:
# All the categories that are possible
rating_categories = ['G - All Ages', 'PG - Children', 'PG-13 - Teens 13 or older', 'R - 17+ (violence & profanity)', 
                     'R+ - Mild Nudity', 'UNKNOWN']

rating_encoded = list() # Empty list that is going to be added too

# Iterate through each row and find append to a list
for i, current_row in anime_df.iterrows():
    if (current_row['Rating'] == rating_categories[0]):
        rating_encoded.append(0)
    elif (current_row['Rating'] == rating_categories[1]):
        rating_encoded.append(1)
    elif (current_row['Rating'] == rating_categories[2]):
        rating_encoded.append(2)
    elif (current_row['Rating'] == rating_categories[3]):
        rating_encoded.append(3)   
    elif (current_row['Rating'] == rating_categories[4]):
        rating_encoded.append(4)
    elif (current_row['Rating'] == rating_categories[5]):
        rating_encoded.append(5)

# Add the list as a column and drop the old ratings columns as it is not needed
anime_df['Rating Encoded'] = rating_encoded
anime_df = anime_df.drop(columns = 'Rating')

anime_df.head(5)

,Score,Genres,Type,Episodes,Licensors,Source,Rating Encoded
Name,,,,,,,
Cowboy Bebop,21.875,"Action, Award Winning, Sci-Fi",TV,1.30,"Funimation, Bandai Entertainment",Original,3
Cowboy Bebop: Tengoku no Tobira,20.950,"Action, Sci-Fi",Movie,0.05,Sony Pictures Entertainment,Original,3
Trigun,20.550,"Action, Adventure, Sci-Fi",TV,1.30,"Funimation, Geneon Entertainment USA",Manga,2
Witch Hunter Robin,18.125,"Action, Drama, Mystery, Supernatural",TV,1.30,"Funimation, Bandai Entertainment",Original,2
Bouken Ou Beet,17.350,"Adventure, Fantasy, Supernatural",TV,2.60,Illumitoon Entertainment,Manga,1


### Total genres
I need to get all of the genres to see how many unique ones there are. Depending on how many genres there are that is how many dimensions we will be adding to our dataframe.

In [6]:
total_genres = list()

for i, current_row in anime_df.iterrows():
    # Need to put the genres in a list
    split_genres = current_row['Genres'].split(',')
    
    # Iterate through the genres in the the row
    for g in range (0, len(split_genres)):
        # Get rid of any leading white space
        split_genres[g] = split_genres[g].strip()
        
        # Add the unique ones
        if (split_genres[g] not in total_genres):
            total_genres.append(split_genres[g])

print("All genres: " + str(total_genres))
print("Number of genres: " + str(len(total_genres)))

All genres: ['Action', 'Award Winning', 'Sci-Fi', 'Adventure', 'Drama', 'Mystery', 'Supernatural', 'Fantasy', 'Comedy', 'Romance', 'Suspense', 'Sports', 'Ecchi', 'Gourmet', 'Avant Garde', 'Horror', 'Slice of Life', 'Girls Love', 'Boys Love', 'UNKNOWN', 'Erotica']
Number of genres: 21


### Total licensors
I need to get all of the licensors to see how many unique ones there are. Depending on how many licensors there are that is how many dimensions we will be adding to our dataframe.

In [7]:
total_licensors = list()

for i, current_row in anime_df.iterrows():
    # Need to put the licensors in a list
    split_licensors = current_row['Licensors'].split(',')
    
    # Iterate through the licensors in the the row
    for g in range (0, len(split_licensors)):
        # Get rid of any leading white space
        split_licensors[g] = split_licensors[g].strip()
        
        # Add the unique ones
        if (split_licensors[g] not in total_licensors):
            total_licensors.append(split_licensors[g])

print("All licensors: " + str(total_licensors))
print("Number of licensors: " + str(len(total_licensors)))

All licensors: ['Funimation', 'Bandai Entertainment', 'Sony Pictures Entertainment', 'Geneon Entertainment USA', 'Illumitoon Entertainment', 'VIZ Media', 'Discotek Media', 'Nozomi Entertainment', 'ADV Films', 'GKIDS', 'Manga Entertainment', 'Media Blasters', 'NYAV Post', 'Aniplex of America', 'AnimEigo', 'Sentai Filmworks', 'UNKNOWN', 'Kadokawa Pictures USA', '4Kids Entertainment', 'Maiden Japan', 'Central Park Media', 'Flatiron Film Company', 'Tokyopop', 'Disney Platform Distribution', 'Harmony Gold', 'Synch-Point', 'Nelvana', 'NIS America', 'Inc.', 'Bandai Visual USA', 'AN Entertainment', 'Urban Vision', 'DreamWorks', 'Warner Bros. Pictures', 'DiC Entertainment', 'Enoki Films', 'Saban Entertainment', 'Crimson Star Media', 'Super Techno Arts', 'Kitty Media', 'Anchor Bay Films', 'Crunchyroll', 'Bandai', 'Eleven Arts', 'Miramax Films', 'Streamline Pictures', 'Shout! Factory', 'The Pokemon Company International', 'Voyager Entertainment', 'Konami Cross Media NY', 'Anime Midstream', 'Hasbr

### Total type
I need to get all of the sources to see how many unique ones there are. Depending on how many sources there are that is how many dimensions we will be adding to our dataframe.

In [8]:
total_types = list(anime_df['Type'].unique())

print("All types: " + str(total_types))
print("Number of types: " + str(len(total_types)))

All types: ['TV', 'Movie', 'OVA', 'Special', 'ONA', 'Music', 'UNKNOWN']
Number of types: 7


### Total sources
I need to get all of the types to see how many unique ones there are. Depending on how many types there are that is how many dimensions we will be adding to our dataframe.

In [9]:
total_sources = list(anime_df['Source'].unique())

print("All types: " + str(total_sources))
print("Number of types: " + str(len(total_sources)))

All types: ['Original', 'Manga', 'Light novel', 'Visual novel', '4-koma manga', 'Novel', 'Other', 'Game', 'Picture book', 'Unknown', 'Book', 'Music', 'Card game', 'Radio', 'Mixed media', 'Web manga', 'Web novel']
Number of types: 17


### One hot encoding

#### Genre
I needed to create a new column for each genre so that I can cluster it properly later. I would have used the sklearn library, but it would have created a unique one for combination. This way of doing it manually makes it easier to analyze.

In [10]:
# Add a column for each genre and set it to 0
for genre in total_genres:
    anime_df[genre] = 0
    
for i, current_row in anime_df.iterrows():
    split_genres = current_row['Genres'].split(',')
    
    for f in range (0, len(split_genres)):
        split_genres[f] = split_genres[f].strip()
        
        # If found, set that genre column to 1
        if (split_genres[f] in split_genres):
            anime_df.at[i, split_genres[f]] += 1

# Get rid of the original column
anime_df = anime_df.drop(columns = 'Genres')

anime_df.head(5)

,Score,Type,Episodes,Licensors,Source,Rating Encoded,Action,Award Winning,Sci-Fi,Adventure,Drama,Mystery,Supernatural,Fantasy,Comedy,Romance,Suspense,Sports,Ecchi,Gourmet,Avant Garde,Horror,Slice of Life,Girls Love,Boys Love,UNKNOWN,Erotica
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,21.875,TV,1.30,"Funimation, Bandai Entertainment",Original,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cowboy Bebop: Tengoku no Tobira,20.950,Movie,0.05,Sony Pictures Entertainment,Original,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Trigun,20.550,TV,1.30,"Funimation, Geneon Entertainment USA",Manga,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Witch Hunter Robin,18.125,TV,1.30,"Funimation, Bandai Entertainment",Original,2,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bouken Ou Beet,17.350,TV,2.60,Illumitoon Entertainment,Manga,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Licensors
I needed to one hot encode all of the licensors so that I can cluster it more accurately.

In [11]:
# Add a column for each genre and set it to 0
for licensors in total_licensors:
    if (licensors != 'UNKNOWN'):
        anime_df[licensors] = 0
    
for i, current_row in anime_df.iterrows():
    split_licensors = current_row['Licensors'].split(',')
    
    for f in range (0, len(split_licensors)):
        split_licensors[f] = split_licensors[f].strip()
        
        # If found, set that genre column to 1
        if (split_licensors[f] in split_licensors):
            anime_df.at[i, split_licensors[f]] += 1

# Get rid of the original column
anime_df = anime_df.drop(columns = 'Licensors')

anime_df.head(5)

,Score,Type,Episodes,Source,Rating Encoded,Action,Award Winning,Sci-Fi,Adventure,Drama,Mystery,Supernatural,Fantasy,Comedy,Romance,Suspense,Sports,Ecchi,Gourmet,Avant Garde,Horror,Slice of Life,Girls Love,Boys Love,UNKNOWN,Erotica,Funimation,Bandai Entertainment,Sony Pictures Entertainment,Geneon Entertainment USA,Illumitoon Entertainment,VIZ Media,Discotek Media,Nozomi Entertainment,ADV Films,GKIDS,Manga Entertainment,Media Blasters,NYAV Post,Aniplex of America,AnimEigo,Sentai Filmworks,Kadokawa Pictures USA,4Kids Entertainment,Maiden Japan,Central Park Media,Flatiron Film Company,Tokyopop,Disney Platform Distribution,Harmony Gold,Synch-Point,Nelvana,NIS America,Inc.,Bandai Visual USA,AN Entertainment,Urban Vision,DreamWorks,Warner Bros. Pictures,DiC Entertainment,Enoki Films,Saban Entertainment,Crimson Star Media,Super Techno Arts,Kitty Media,Anchor Bay Films,Crunchyroll,Bandai,Eleven Arts,Miramax Films,Streamline Pictures,Shout! Factory,The Pokemon Company International,Voyager Entertainment,Konami Cross Media NY,Anime Midstream,Hasbro,Saban Brands,Capcom,Hirameki International,Cinelicious Pics,Arts Magic,Frontier Works,Pied Piper,Mill Creek Entertainment,Marvel Entertainment,Nintendo of America,Dentsu Entertainment USA,Cookie Jar Entertainment,ADK Emotions NY,Ascendent Animation,Ketchup Entertainment,Kadokawa,Ponycan USA,Kuma Holdings,bilibili,NBCUniversal Entertainment Japan,Bandai Namco Games,Critical Mass Video,iQIYI,Muse Communication,Travel Compass
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,21.875,TV,1.30,Original,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cowboy Bebop: Tengoku no Tobira,20.950,Movie,0.05,Original,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Trigun,20.550,TV,1.30,Manga,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Witch Hunter Robin,18.125,TV,1.30,Original,2,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bouken Ou Beet,17.350,TV,2.60,Manga,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Type
I needed to one hot encode all of the types so that I can cluster it more accurately.

In [12]:
# Add a column for each type and set it to 0
for c_type in total_types:
    if (c_type != 'UNKNOWN'):
        anime_df[c_type] = 0
        
for i, current_row in anime_df.iterrows():
    stripped_type = current_row['Type']
    
    for f in range (0, len(total_types)):
        if (stripped_type == total_types[f]):
            anime_df.at[i, total_types[f]] += 1
            
# Get rid of the original column
anime_df = anime_df.drop(columns = 'Type')
        
anime_df.head(5)

,Score,Episodes,Source,Rating Encoded,Action,Award Winning,Sci-Fi,Adventure,Drama,Mystery,Supernatural,Fantasy,Comedy,Romance,Suspense,Sports,Ecchi,Gourmet,Avant Garde,Horror,Slice of Life,Girls Love,Boys Love,UNKNOWN,Erotica,Funimation,Bandai Entertainment,Sony Pictures Entertainment,Geneon Entertainment USA,Illumitoon Entertainment,VIZ Media,Discotek Media,Nozomi Entertainment,ADV Films,GKIDS,Manga Entertainment,Media Blasters,NYAV Post,Aniplex of America,AnimEigo,Sentai Filmworks,Kadokawa Pictures USA,4Kids Entertainment,Maiden Japan,Central Park Media,Flatiron Film Company,Tokyopop,Disney Platform Distribution,Harmony Gold,Synch-Point,Nelvana,NIS America,Inc.,Bandai Visual USA,AN Entertainment,Urban Vision,DreamWorks,Warner Bros. Pictures,DiC Entertainment,Enoki Films,Saban Entertainment,Crimson Star Media,Super Techno Arts,Kitty Media,Anchor Bay Films,Crunchyroll,Bandai,Eleven Arts,Miramax Films,Streamline Pictures,Shout! Factory,The Pokemon Company International,Voyager Entertainment,Konami Cross Media NY,Anime Midstream,Hasbro,Saban Brands,Capcom,Hirameki International,Cinelicious Pics,Arts Magic,Frontier Works,Pied Piper,Mill Creek Entertainment,Marvel Entertainment,Nintendo of America,Dentsu Entertainment USA,Cookie Jar Entertainment,ADK Emotions NY,Ascendent Animation,Ketchup Entertainment,Kadokawa,Ponycan USA,Kuma Holdings,bilibili,NBCUniversal Entertainment Japan,Bandai Namco Games,Critical Mass Video,iQIYI,Muse Communication,Travel Compass,TV,Movie,OVA,Special,ONA,Music
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,21.875,1.30,Original,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Cowboy Bebop: Tengoku no Tobira,20.950,0.05,Original,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Trigun,20.550,1.30,Manga,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Witch Hunter Robin,18.125,1.30,Original,2,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Bouken Ou Beet,17.350,2.60,Manga,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Source
I needed to one hot encode all of the sources so that I can cluster it more accurately.

In [13]:
# Add a column for each type and set it to 0
for c_sources in total_sources:
    if (c_sources != 'UNKNOWN'):
        anime_df[c_sources] = 0
        
for i, current_row in anime_df.iterrows():
    stripped_sources = current_row['Source']
    
    for f in range (0, len(total_sources)):
        if (stripped_sources == total_sources[f]):
            anime_df.at[i, total_sources[f]] += 1
            
# Get rid of the original column
anime_df = anime_df.drop(columns = 'Source')
        
anime_df.head(5)

,Score,Episodes,Rating Encoded,Action,Award Winning,Sci-Fi,Adventure,Drama,Mystery,Supernatural,Fantasy,Comedy,Romance,Suspense,Sports,Ecchi,Gourmet,Avant Garde,Horror,Slice of Life,Girls Love,Boys Love,UNKNOWN,Erotica,Funimation,Bandai Entertainment,Sony Pictures Entertainment,Geneon Entertainment USA,Illumitoon Entertainment,VIZ Media,Discotek Media,Nozomi Entertainment,ADV Films,GKIDS,Manga Entertainment,Media Blasters,NYAV Post,Aniplex of America,AnimEigo,Sentai Filmworks,Kadokawa Pictures USA,4Kids Entertainment,Maiden Japan,Central Park Media,Flatiron Film Company,Tokyopop,Disney Platform Distribution,Harmony Gold,Synch-Point,Nelvana,NIS America,Inc.,Bandai Visual USA,AN Entertainment,Urban Vision,DreamWorks,Warner Bros. Pictures,DiC Entertainment,Enoki Films,Saban Entertainment,Crimson Star Media,Super Techno Arts,Kitty Media,Anchor Bay Films,Crunchyroll,Bandai,Eleven Arts,Miramax Films,Streamline Pictures,Shout! Factory,The Pokemon Company International,Voyager Entertainment,Konami Cross Media NY,Anime Midstream,Hasbro,Saban Brands,Capcom,Hirameki International,Cinelicious Pics,Arts Magic,Frontier Works,Pied Piper,Mill Creek Entertainment,Marvel Entertainment,Nintendo of America,Dentsu Entertainment USA,Cookie Jar Entertainment,ADK Emotions NY,Ascendent Animation,Ketchup Entertainment,Kadokawa,Ponycan USA,Kuma Holdings,bilibili,NBCUniversal Entertainment Japan,Bandai Namco Games,Critical Mass Video,iQIYI,Muse Communication,Travel Compass,TV,Movie,OVA,Special,ONA,Music,Original,Manga,Light novel,Visual novel,4-koma manga,Novel,Other,Game,Picture book,Unknown,Book,Card game,Radio,Mixed media,Web manga,Web novel
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,21.875,1.30,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cowboy Bebop: Tengoku no Tobira,20.950,0.05,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Trigun,20.550,1.30,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Witch Hunter Robin,18.125,1.30,2,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bouken Ou Beet,17.350,2.60,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
km = KMeans(n_clusters = 15)
km.fit(anime_df)


# for i, current_row in anime_df.iterrows():
#     current_row = list(current_row)
#     print(current_row)

KMeans(n_clusters=15)